<a href="https://colab.research.google.com/github/adamserag1/Interpretability-for-VRDU-models/blob/main/finetuning/BROS-FUNSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BROS

## Imports etc.

In [1]:
# UNCOMMNET FOR USE IN COLAB
!git clone https://github.com/adamserag1/Interpretability-for-VRDU-models.git

Cloning into 'Interpretability-for-VRDU-models'...
remote: Enumerating objects: 712, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 712 (delta 41), reused 32 (delta 14), pack-reused 634 (from 1)
Receiving objects: 100% (712/712), 16.02 MiB | 20.98 MiB/s, done.
Resolving deltas: 100% (377/377), done.


In [1]:
!pip install transformers torch datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.3 MB/s eta 0:00:00


In [2]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.7.0 requires fsspec==2025.7.0, but you have fsspec 2025.3.0 which is incompatible.


In [3]:
import torch
from PIL import Image,ImageDraw, ImageFont
from datasets import load_dataset
import pandas as pd
import evaluate
from transformers import BrosProcessor, BrosForTokenClassification, BrosSpadeEEForTokenClassification, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Data

In [4]:
funsd = load_dataset("nielsr/funsd", trust_remote_code=True)
label_list = funsd["train"].features["ner_tags"].feature.names
id2label = {v:k for v,k in enumerate(label_list)}
label2id = {k:v for v,k in enumerate(label_list)}

print(funsd["train"]["words"][0])
print(funsd["train"]["ner_tags"][0])
id_0_ner_tags = [id2label[id] for id in funsd["train"]["ner_tags"][0]]
print(id_0_ner_tags)

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'nielsr/funsd' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'nielsr/funsd' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this sec

README.md:   0%|          | 0.00/755 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/4.38M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/149 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50 [00:00<?, ? examples/s]

['R&D', ':', 'Suggestion:', 'Date:', 'Licensee', 'Yes', 'No', '597005708', 'R&D', 'QUALITY', 'IMPROVEMENT', 'SUGGESTION/', 'SOLUTION', 'FORM', 'Name', '/', 'Phone', 'Ext.', ':', 'M.', 'Hamann', 'P.', 'Harper,', 'P.', 'Martinez', '9/', '3/', '92', 'R&D', 'Group:', 'J.', 'S.', 'Wigand', 'Supervisor', '/', 'Manager', 'Discontinue', 'coal', 'retention', 'analyses', 'on', 'licensee', 'submitted', 'product', 'samples', '(Note', ':', 'Coal', 'Retention', 'testing', 'is', 'not', 'performed', 'by', 'most', 'licensees.', 'Other', 'B&W', 'physical', 'measurements', 'as', 'ends', 'stability', 'and', 'inspection', 'for', 'soft', 'spots', 'in', 'ciparettes', 'are', 'thought', 'to', 'be', 'sufficient', 'measures', 'to', 'assure', 'cigarette', 'physical', 'integrity.', 'The', 'proposed', 'action', 'will', 'increase', 'laboratory', 'productivity', '.', ')', 'Suggested', 'Solutions', '(s)', ':', 'Delete', 'coal', 'retention', 'from', 'the', 'list', 'of', 'standard', 'analyses', 'performed', 'on', 'licen

In [5]:
processor = BrosProcessor.from_pretrained("naver-clova-ocr/bros-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("naver-clova-ocr/bros-base-uncased")
model = BrosForTokenClassification.from_pretrained("naver-clova-ocr/bros-base-uncased", num_labels=len(label_list), id2label=id2label, label2id=label2id)
# def freeze_bros_layers(model,
#                        num_encoder_layers_to_freeze = 5,
#                        freeze_embeddings= True):
#     """
#     Args
#     ----
#     model  : your BrosForDocumentClassification instance
#     num_encoder_layers_to_freeze : freeze layers 0 … N-1 (default 6 of 12)
#     freeze_embeddings            : also freeze word/2-D/pos embeddings
#     """
#     # 1) embeddings
#     if freeze_embeddings:
#         for p in model.bros.embeddings.parameters():
#             p.requires_grad = False

#     # 2) first N encoder layers
#     for layer in model.bros.encoder.layer[:num_encoder_layers_to_freeze]:
#         for p in layer.parameters():
#             p.requires_grad = False

#     # 3) report
#     tot = sum(p.numel() for p in model.parameters())
#     train = sum(p.numel() for p in model.parameters() if p.requires_grad)
#     print(f" Froze embeddings={freeze_embeddings}, "
#           f"layers 0-{num_encoder_layers_to_freeze-1}. "
#           f"Trainable params: {train/1e6:.1f} M / {tot/1e6:.1f} M")

# freeze_bros_layers(model, num_encoder_layers_to_freeze=6, freeze_embeddings=True)
model.to(device)

vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BrosForTokenClassification were not initialized from the model checkpoint at naver-clova-ocr/bros-base-uncased and are newly initialized: ['bros.bbox_embeddings.bbox_projection.weight', 'bros.bbox_embeddings.bbox_sinusoid_emb.x_pos_emb.inv_freq', 'bros.bbox_embeddings.bbox_sinusoid_emb.y_pos_emb.inv_freq', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

BrosForTokenClassification(
  (bros): BrosModel(
    (embeddings): BrosTextEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (bbox_embeddings): BrosBboxEmbeddings(
      (bbox_sinusoid_emb): BrosPositionalEmbedding2D(
        (x_pos_emb): BrosPositionalEmbedding1D()
        (y_pos_emb): BrosPositionalEmbedding1D()
      )
      (bbox_projection): Linear(in_features=192, out_features=64, bias=False)
    )
    (encoder): BrosEncoder(
      (layer): ModuleList(
        (0-11): 12 x BrosLayer(
          (attention): BrosAttention(
            (self): BrosSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Lin

In [6]:
sample = funsd["train"][66]
image = Image.open(sample["image_path"]).convert("RGB")
print(sample['words'])

KeyError: 'image_path'

In [6]:
def normalize_bbox(bbox, width, height):
    return [
        int(1000 * (bbox[0] / width)),
        int(1000 * (bbox[1] / height)),
        int(1000 * (bbox[2] / width)),
        int(1000 * (bbox[3] / height)),
    ]
# def tokenize_words(batch):
#     enc = tokenizer(
#         batch["words"],
#         is_split_into_words=True,
#         truncation=True,
#         padding="max_length",
#         max_length=512,
#     )
#     batch_boxes, batch_init, batch_subs = [], [], []

#     for idx, (bboxes, img, word_tags) in enumerate(zip(batch["bboxes"], batch["image"], batch["ner_tags"])):
#         w, h = img.size
#         norm_boxes = [normalize_bbox(bb, w, h) for bb in bboxes]

#         page_boxes, page_i, page_s = [], [], []
#         for word_id in enc.word_ids(batch_index=idx):
#             if word_id is None:
#                 page_boxes.append([0,0,0,0])
#                 page_i.append(-100)
#                 page_s.append(-100)
#             else:
#                 tag = id2label[word_tags[word_id]]
#                 page_boxes.append(norm_boxes[word_id])
#                 # B- tag → initial, I- tag → subsequent, O → neither
#                 if   tag.startswith("B-"): page_i.append(1); page_s.append(0)
#                 elif tag.startswith("I-"): page_i.append(0); page_s.append(1)
#                 else:                     page_i.append(0); page_s.append(0)

#         batch_boxes.append(page_boxes)
#         batch_init.append(page_i)
#         batch_subs.append(page_s)

#     enc["bbox"]                    = batch_boxes
#     enc["initial_token_labels"]    = batch_init
#     enc["subsequent_token_labels"] = batch_subs
#     return enc


def tokenize_words(batch):
  encodings = tokenizer(
    batch["words"],
    is_split_into_words=True,
    truncation=True,
    padding="max_length",
    max_length=512,
    return_tensors="pt"
  )

  batch_normalized_bboxes, encoded_labels = [], []
  for idx, (bboxes, img, labels) in enumerate(zip(batch["bboxes"], batch["image"], batch["ner_tags"])):
    width, height = img.size
    normalized_bboxes = [normalize_bbox(bbox, width, height) for bbox in bboxes]

    # Align boxes to sub words
    aligned_boxes, aligned_labels = [], []
    for word_id in encodings.word_ids(batch_index=idx):
      if word_id is None:
        aligned_boxes.append([0, 0, 0, 0])
        aligned_labels.append(-100)
      else:
        aligned_boxes.append(normalized_bboxes[word_id])
        aligned_labels.append(labels[word_id])

    batch_normalized_bboxes.append(aligned_boxes)
    encoded_labels.append(aligned_labels)

  encodings['bbox'] = batch_normalized_bboxes
  encodings['labels'] = encoded_labels

  return encodings

In [7]:
train_dataset = funsd["train"].map(tokenize_words, batched=True, remove_columns=funsd["train"].column_names)
val_dataset = funsd["test"].map(tokenize_words, batched=True, remove_columns=funsd["train"].column_names)

train_dataset.set_format("torch")
val_dataset.set_format("torch")

Map:   0%|          | 0/149 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

## Finetuning

In [32]:
!rm -r /content/bros-funsd-finetuned-frz/

In [8]:
from transformers import TrainingArguments, Trainer, DefaultDataCollator
training_args = TrainingArguments(
    output_dir="./bros-funsd-finetuned",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=100,
    save_strategy="epoch",
    optim="adamw_torch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to = 'none',
    metric_for_best_model="eval_f1",
)
# training_args = TrainingArguments(
#     output_dir="./bros-spadeee-funsd",

#     # — evaluation & checkpointing —
#     eval_strategy="steps",
#     eval_steps=200,                         # validate every 200 update steps
#     save_strategy="steps",
#     save_steps=200,                         # checkpoint every 200 update steps
#     logging_steps=200,                      # log metrics every 200 update steps
#     load_best_model_at_end=True,
#     metric_for_best_model="init_f1",        # pick the checkpoint with best boundary-F1
#     greater_is_better=True,

#     # — optimization —
#     num_train_epochs=30,
#     per_device_train_batch_size=8,          # batch of 8 pages / device
#     gradient_accumulation_steps=4,          # → effective batch of 32 pages
#     learning_rate=2e-5,
#     warmup_ratio=0.1,                       # first 10% of steps
#     weight_decay=0.01,
#     lr_scheduler_type="linear",
#     max_grad_norm=1.0,
#     fp16=True,                              # mixed-precision

#     # — SpaDE-EE labels —
#     label_names=["initial_token_labels", "subsequent_token_labels"],

#     # — bookkeeping —
#     report_to="none",
#     push_to_hub=False,
# )
# Data collator
data_collator = DefaultDataCollator(return_tensors="pt")

In [9]:
metric = evaluate.load("seqeval")
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions, axis=-1)

  true_preds = [
      [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]
  true_labels = [
      [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]
  results = metric.compute(predictions=true_preds, references=true_labels)

  return {
      "precision": results["overall_precision"],
      "recall": results["overall_recall"],
      "f1": results["overall_f1"],
      "accuracy": results["overall_accuracy"],
  }

# def compute_boundary_metrics(p):
#   (init_logits, subs_logits), (init_labels, subs_labels) = p

#   # mask out padding
#   mask = init_labels.view(-1) != -100
#   i_preds = init_logits.argmax(-1).view(-1)[mask]
#   s_preds = subs_logits .argmax(-1).view(-1)[mask]
#   i_golds = init_labels.view(-1)[mask]
#   s_golds = subs_labels .view(-1)[mask]

#   pi, ri, fi, _ = precision_recall_fscore_support(i_golds, i_preds, average="binary", zero_division=0)
#   ps, rs, fs, _ = precision_recall_fscore_support(s_golds, s_preds, average="binary", zero_division=0)

#   return {
#     "init_prec": pi, "init_rec": ri, "init_f1": fi,
#     "sub_prec" : ps, "sub_rec" : rs, "sub_f1" : fs,
#   }

In [10]:
from transformers import EarlyStoppingCallback
early_stop = EarlyStoppingCallback(
    early_stopping_patience   = 6,
    early_stopping_threshold = 0.0
)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1731: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.708234,0.000000,0.000000,0.000000,0.294141
2,No log,1.646404,0.000000,0.000000,0.000000,0.337197
3,No log,1.650882,0.000000,0.000000,0.000000,0.344373
4,No log,1.623056,0.028269,0.002535,0.004653,0.388140
5,No log,1.642729,0.036072,0.005703,0.009850,0.367558
6,No log,1.550355,0.120405,0.071610,0.089807,0.387430
7,No log,1.473930,0.212310,0.216413,0.214342,0.430723
8,No log,1.341477,0.305260,0.318124,0.311559,0.462582
9,No log,1.255265,0.377409,0.359949,0.368472,0.528507
10,No log,1.149174,0.433800,0.383080,0.406865,0.582762


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1731: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1731: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.11/dist

TrainOutput(global_step=1900, training_loss=0.19552582925871798, metrics={'train_runtime': 944.7699, 'train_samples_per_second': 15.771, 'train_steps_per_second': 2.011, 'total_flos': 3921093177446400.0, 'train_loss': 0.19552582925871798, 'epoch': 100.0})

In [17]:
best_model = trainer.model
best_model.push_to_hub("adamadam111/bros-funsd-finetuned-2.0")
tokenizer.save_pretrained("adamadam111/bros-funsd-finetuned-2.0")
tokenizer.push_to_hub("adamadam111/bros-funsd-finetuned-2.0")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/adamadam111/bros-funsd-finetuned-2.0/commit/b24c2800d1a378961489f28fed663a5a2559fc0e', commit_message='Upload tokenizer', commit_description='', oid='b24c2800d1a378961489f28fed663a5a2559fc0e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/adamadam111/bros-funsd-finetuned-2.0', endpoint='https://huggingface.co', repo_type='model', repo_id='adamadam111/bros-funsd-finetuned-2.0'), pr_revision=None, pr_num=None)

In [15]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `adam` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `adam`


In [ ]:
'''
Model available on huggingface hub: https://huggingface.co/adamadam111/bros-finetuned-funsd
'''

## Inference

Upload a test form as 'test.jpg' for inference

In [ ]:
!pip install pytesseract

In [ ]:
import pytesseract, cv2

tokenizer = AutoTokenizer.from_pretrained("adamadam111/bros-funsd-finetuned",do_lower_case=True)
model = BrosForTokenClassification.from_pretrained("adamadam111/bros-funsd-finetuned")

image_path = "test.jpg"
img = cv2.imread(image_path)
ocr_data = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)

In [ ]:
words, bboxes = [], []
for i in range(len(ocr_data["text"])):
  word = ocr_data["text"][i].strip()
  if word == "":
    continue
  x, y, w, h = (ocr_data[k][i] for k in ["left","top","width","height"])
  words.append(word)
  bboxes.append([x, y, x+w, y+h])


In [ ]:
height, width = img.shape[:2]
boxes_1000 = [normalize_bbox(bbox, width, height) for bbox in bboxes]


In [ ]:
enc = tokenizer(words,
          is_split_into_words=True,
          return_tensors="pt",
          padding="max_length",
          truncation=True,
          max_length=512)

aligned_boxes = []
word_ids = enc.word_ids()
for wi in word_ids:
  if wi is None:
    aligned_boxes.append([0,0,0,0])
  else:
    aligned_boxes.append(boxes_1000[wi])

enc["bbox"] = torch.tensor([aligned_boxes])


In [ ]:
def unnormalize_box(box_1000, w, h):
  return [
    int(box_1000[0] * w / 1000),
    int(box_1000[1] * h / 1000),
    int(box_1000[2] * w / 1000),
    int(box_1000[3] * h / 1000),
  ]

def iob_to_label(tag):
  print(tag)
  core = tag[2:] if tag.startswith(("B-","I-")) else tag
  return core.lower() if core else "other"

label2color = {
  "question": "blue",
  "answer":   "green",
  "header":   "orange",
  "other":    "violet",
}

In [ ]:
outputs   = model(**enc)
pred_ids  = outputs.logits.argmax(dim=-1).squeeze().tolist()
word_ids  = enc.word_ids()

true_preds, true_boxes = [], []
prev_word = None
for pid, wid, box in zip(pred_ids, word_ids, aligned_boxes):
  if wid is None or wid == prev_word:
    continue
  true_preds.append(id2label[pid])
  true_boxes.append(unnormalize_box(box, width, height))
  prev_word = wid

true_preds = [id2label[p] for p, l in zip(pred_ids, word_ids) if l != -100]
true_boxes = [unnormalize_box(b, width, height) for b, l in zip(aligned_boxes, word_ids) if l != -100]

img_pil = Image.fromarray(img)
draw = ImageDraw.Draw(img_pil)
font = ImageFont.load_default()

for pred, box in zip(true_preds, true_boxes):
  label = iob_to_label(pred)
  color = label2color.get(label, "red")
  draw.rectangle(box, outline=color, width=2)
  draw.text((box[0] + 3, box[1] - 10), text=label, fill=color, font=font)

img_pil
#img_pil.save('/content/test_ann.jpg')